<a href="https://colab.research.google.com/github/AhmadJamshaid/AgenticAI_Workshop/blob/main/GMClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install python-chess tensorflow pandas numpy scikit-learn

In [12]:
!wget https://github.com/official-stockfish/Stockfish/releases/download/sf_16.1/stockfish-ubuntu-x86-64-avx2.tar


--2025-11-30 21:55:19--  https://github.com/official-stockfish/Stockfish/releases/download/sf_16.1/stockfish-ubuntu-x86-64-avx2.tar
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/20976138/2c8919f2-0a02-4f38-ab23-d1919a99521d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-30T22%3A35%3A21Z&rscd=attachment%3B+filename%3Dstockfish-ubuntu-x86-64-avx2.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-30T21%3A34%3A47Z&ske=2025-11-30T22%3A35%3A21Z&sks=b&skv=2018-11-09&sig=AVlWUQk4dc0w58f5M9AqpEnF%2BbVVBrEO6NVXbe3AbFI%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDU0MTUxOSwibmJmIjoxNzY0NTM5NzE5L

In [13]:
!tar -xvf stockfish-ubuntu-x86-64-avx2.tar


stockfish/
stockfish/Copying.txt
stockfish/CONTRIBUTING.md
stockfish/wiki/
stockfish/wiki/Home.md
stockfish/wiki/Useful-data.md
stockfish/wiki/Governance-and-responsibilities.md
stockfish/wiki/Stockfish-FAQ.md
stockfish/wiki/Regression-Tests.md
stockfish/wiki/Terminology.md
stockfish/wiki/_Footer.md
stockfish/wiki/UCI-&-Commands.md
stockfish/wiki/Advanced-topics.md
stockfish/wiki/Developers.md
stockfish/wiki/Compiling-from-source.md
stockfish/wiki/Download-and-usage.md
stockfish/README.md
stockfish/src/
stockfish/src/benchmark.cpp
stockfish/src/syzygy/
stockfish/src/syzygy/tbprobe.cpp
stockfish/src/syzygy/tbprobe.h
stockfish/src/uci.cpp
stockfish/src/tt.h
stockfish/src/timeman.cpp
stockfish/src/benchmark.h
stockfish/src/position.cpp
stockfish/src/types.h
stockfish/src/evaluate.cpp
stockfish/src/perft.h
stockfish/src/search.h
stockfish/src/tt.cpp
stockfish/src/movegen.h
stockfish/src/movepick.h
stockfish/src/thread.h
stockfish/src/Makefile
stockfish/src/ucioption.h
stockfish/src/uciopti

In [14]:
!chmod +x stockfish/stockfish-ubuntu-x86-64-avx2


In [15]:
STOCKFISH_PATH = "/content/stockfish/stockfish-ubuntu-x86-64-avx2"


In [16]:
import chess
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
print("Loaded:", engine.id)
engine.quit()


Loaded: {'name': 'Stockfish 16.1', 'author': 'the Stockfish developers (see AUTHORS file)'}


In [17]:
!ls /content/

Carlsen.pgn   Giri..pgn     stockfish-ubuntu-x86-64-avx2.tar
Caruana.pgn   Nakamura.pgn  stockfish-ubuntu-x86-64-avx2.tar.1
drive	      sample_data
Firouzja.pgn  stockfish


In [ ]:
# =============================================
# 1️⃣ Install required libraries
# =============================================
!pip install python-chess tensorflow pandas numpy scikit-learn matplotlib

import chess
import chess.pgn
import chess.engine
import pandas as pd
import numpy as np
import os
import random
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# =============================================
# 2️⃣ Stockfish path
# =============================================
STOCKFISH_PATH = "/content/stockfish/stockfish-ubuntu-x86-64-avx2"
!chmod +x {STOCKFISH_PATH}

# =============================================
# 3️⃣ PGN files paths for 5 GMs
# =============================================
PGN_FILES = {
    "Carlsen": "/content/Carlsen.pgn",
    "Nakamura": "/content/Nakamura.pgn",
    "Caruana": "/content/Caruana.pgn",
    "Giri": "/content/Giri.pgn",
    "Firouzja": "/content/Firouzja.pgn"
}

# =============================================
# 4️⃣ Feature extraction functions
# =============================================
def compute_material(board):
    piece_values = {chess.PAWN:1, chess.KNIGHT:3, chess.BISHOP:3,
                    chess.ROOK:5, chess.QUEEN:9, chess.KING:0}
    white = sum([piece_values[p.piece_type] for p in board.piece_map().values() if p.color==chess.WHITE])
    black = sum([piece_values[p.piece_type] for p in board.piece_map().values() if p.color==chess.BLACK])
    return white - black

def compute_mobility(board):
    return len(list(board.legal_moves))

def compute_center_control(board):
    center_squares = [chess.D4, chess.D5, chess.E4, chess.E5]
    control = 0
    for sq in center_squares:
        attackers_white = board.attackers(chess.WHITE, sq)
        attackers_black = board.attackers(chess.BLACK, sq)
        control += len(attackers_white) - len(attackers_black)
    return control

def compute_king_safety(board):
    kings = {chess.WHITE: board.king(chess.WHITE), chess.BLACK: board.king(chess.BLACK)}
    safety = 0
    for color, sq in kings.items():
        if sq is not None:
            attackers = len(board.attackers(not color, sq))
            safety += -attackers if color==chess.WHITE else attackers
    return safety

def compute_pawn_structure(board):
    pawns = board.pieces(chess.PAWN, chess.WHITE) | board.pieces(chess.PAWN, chess.BLACK)
    doubled = 0
    for file in range(8):
        count = sum([chess.square_file(p)==file for p in pawns])
        if count>1:
            doubled += count-1
    return -doubled

def compute_attack_score(board):
    score = 0
    for move in board.legal_moves:
        if board.is_capture(move):
            score += 1
    return score

# Optional faster mode: reduce Stockfish depth
def compute_eval(board, depth=5):
    try:
        with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
            info = engine.analyse(board, chess.engine.Limit(depth=depth))
            val = info["score"].white().score(mate_score=10000)
            return val if val is not None else 0
    except:
        return 0

# =============================================
# 5️⃣ Batch processing with resume support
# =============================================
BATCH_SIZE = 500  # moves per batch
all_data = []

for gm_name, path in PGN_FILES.items():
    print(f"\nProcessing {gm_name}...")

    # Detect existing batches
    existing_batches = [f for f in os.listdir("/content/") if f.startswith(f"chess_features_batch_{gm_name}_")]
    if existing_batches:
        last_batch = max(existing_batches, key=lambda x: int(x.split("_")[-1].split(".")[0]))
        print(f"Existing batch detected: {last_batch}, skipping previous moves.")
        # Optionally, you could skip the already processed moves in PGN (advanced)

    batch_count = len(existing_batches)
    with open(path) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                color = "White" if board.turn==chess.BLACK else "Black"
                row = [
                    compute_material(board),
                    compute_mobility(board),
                    compute_center_control(board),
                    compute_king_safety(board),
                    compute_pawn_structure(board),
                    compute_attack_score(board),
                    compute_eval(board),
                    color,
                    gm_name
                ]
                all_data.append(row)

                # Save batch
                if len(all_data) % BATCH_SIZE == 0:
                    batch_count += 1
                    temp_df = pd.DataFrame(all_data, columns=[
                        "material","mobility","center","king_safety","pawn_structure",
                        "attack_score","eval","color","GM"])
                    temp_df.to_csv(f"chess_features_batch_{gm_name}_{batch_count}.csv", index=False)
                    print(f"Saved batch {batch_count} for {gm_name}")
                    all_data = []  # reset list for next batch

# Save remaining moves
if all_data:
    batch_count += 1
    temp_df = pd.DataFrame(all_data, columns=[
        "material","mobility","center","king_safety","pawn_structure",
        "attack_score","eval","color","GM"])
    temp_df.to_csv(f"chess_features_batch_{gm_name}_{batch_count}.csv", index=False)
    print(f"Saved final batch {batch_count} for {gm_name}")

# =============================================
# 6️⃣ Combine all batches into final dataset
# =============================================
batch_files = [f for f in os.listdir("/content/") if f.startswith("chess_features_batch_")]
dfs = [pd.read_csv(f) for f in batch_files]
df = pd.concat(dfs).reset_index(drop=True)

# Downsample to 4000 rows per GM
final_dfs = []
for gm in df["GM"].unique():
    gm_df = df[df["GM"]==gm]
    if len(gm_df) > 4000:
        gm_df = gm_df.sample(4000, random_state=42)
    final_dfs.append(gm_df)
df_balanced = pd.concat(final_dfs).reset_index(drop=True)

# Encode color and normalize
df_balanced["color"] = df_balanced["color"].map({"White":0,"Black":1})
num_features = ["material","mobility","center","king_safety","pawn_structure","attack_score","eval","color"]
scaler = StandardScaler()
df_balanced[num_features] = scaler.fit_transform(df_balanced[num_features])

# Save final CSV
df_balanced.to_csv("chess_features_dataset.csv", index=False)
print("✅ Final dataset saved: chess_features_dataset.csv")